# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import gmaps.datasets
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df=('../output_data/cities.csv')
data=pd.read_csv(df)
data

,Unnamed: 0,City,Latitude,Longitude,Max_temp,Humidity,Cloudiness,Wind_speed,Country,Date
0,0,Denver,39.7392,-104.9847,52.00,21,0,1.01,US,1616095856
1,1,Boston,42.3584,-71.0598,46.99,87,100,5.75,US,1616095937
2,2,Los Angeles,34.0522,-118.2437,68.00,52,1,5.41,US,1616096055
3,3,Oregon,44.0001,-120.5014,55.99,39,100,13.00,US,1616096228
4,4,Quito,-0.2299,-78.5249,64.40,63,75,8.05,EC,1616096187
...,...,...,...,...,...,...,...,...,...,...
566,583,Wukari,7.8500,9.7833,83.17,63,100,7.92,NG,1616140671
567,584,Sungai Penuh,-2.0833,101.3833,71.65,93,100,2.77,ID,1616140672
568,585,La Rinconada,37.4873,-5.9826,50.00,53,0,9.22,ES,1616140674
569,586,Moose Factory,51.2585,-80.6094,10.40,67,1,10.36,CA,1616140675


In [3]:
data.count()

Unnamed: 0    571
City          571
Latitude      571
Longitude     571
Max_temp      571
Humidity      571
Cloudiness    571
Wind_speed    571
Country       566
Date          571
dtype: int64

In [4]:
data.dropna(inplace = True) 
del data['Unnamed: 0']
data

,City,Latitude,Longitude,Max_temp,Humidity,Cloudiness,Wind_speed,Country,Date
0,Denver,39.7392,-104.9847,52.00,21,0,1.01,US,1616095856
1,Boston,42.3584,-71.0598,46.99,87,100,5.75,US,1616095937
2,Los Angeles,34.0522,-118.2437,68.00,52,1,5.41,US,1616096055
3,Oregon,44.0001,-120.5014,55.99,39,100,13.00,US,1616096228
4,Quito,-0.2299,-78.5249,64.40,63,75,8.05,EC,1616096187
...,...,...,...,...,...,...,...,...,...
566,Wukari,7.8500,9.7833,83.17,63,100,7.92,NG,1616140671
567,Sungai Penuh,-2.0833,101.3833,71.65,93,100,2.77,ID,1616140672
568,La Rinconada,37.4873,-5.9826,50.00,53,0,9.22,ES,1616140674
569,Moose Factory,51.2585,-80.6094,10.40,67,1,10.36,CA,1616140675


In [5]:
data.count()

City          566
Latitude      566
Longitude     566
Max_temp      566
Humidity      566
Cloudiness    566
Wind_speed    566
Country       566
Date          566
dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
locations = data[['Latitude', 'Longitude']]
weights = data['Humidity']
fig = gmaps.figure()
layer=gmaps.heatmap_layer(locations, weights=weights)
fig.add_layer(layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# , dissipating = False, max_intensity = 75, point_radius = 1

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [18]:
tempa=data.loc[(data['Max_temp']>=65) & (data['Max_temp']<=90)]
tempa

humid=tempa.loc[(tempa['Humidity']>=5) & (tempa['Humidity']<=60)]
humid

windspeed=humid.loc[(humid['Wind_speed']>=5) & (humid['Wind_speed']<=30)]
windspeed

clouds_df=windspeed.loc[(windspeed['Cloudiness']>=5) & (windspeed['Cloudiness']<=80)]

clouds_df

# temp= data[data['Max_temp'] >=65 & data['Max_temp']<=90]

,City,Latitude,Longitude,Max_temp,Humidity,Cloudiness,Wind_speed,Country,Date
10,Hobart,-42.8794,147.3294,69.80,49,20,11.50,AU,1616139813
47,Mahabalipuram,12.6264,80.1722,87.80,55,40,8.05,IN,1616139884
73,Cape Town,-33.9258,18.4232,66.20,59,20,20.71,ZA,1616139684
177,Pindiga,9.9833,10.9333,84.36,52,75,12.30,NG,1616140079
186,Hong Kong,22.2855,114.1577,82.40,54,40,6.91,HK,1616140037
205,Pala,9.3642,14.9046,89.10,37,71,5.48,TD,1616140123
252,Yambio,4.5721,28.3955,88.65,42,50,5.64,SS,1616140197
261,Douentza,14.9951,-2.9517,82.22,8,51,5.82,ML,1616140212
317,Port Pirie,-33.1833,138.0167,82.40,44,67,12.66,AU,1616140301
332,Broken Hill,-31.9500,141.4333,80.60,41,40,20.71,AU,1616140323


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
hotel_df=clouds_df

hotel_df['Hotel Name']=""
hotel_df['Hotel Rating']=""
hotel_df

<ipython-input-30-aeed5dc0eacd>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Name']=""
<ipython-input-30-aeed5dc0eacd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df['Hotel Rating']=""


,City,Latitude,Longitude,Max_temp,Humidity,Cloudiness,Wind_speed,Country,Date,Hotel Name,Hotel Rating
10,Hobart,-42.8794,147.3294,69.80,49,20,11.50,AU,1616139813,,
47,Mahabalipuram,12.6264,80.1722,87.80,55,40,8.05,IN,1616139884,,
73,Cape Town,-33.9258,18.4232,66.20,59,20,20.71,ZA,1616139684,,
177,Pindiga,9.9833,10.9333,84.36,52,75,12.30,NG,1616140079,,
186,Hong Kong,22.2855,114.1577,82.40,54,40,6.91,HK,1616140037,,
205,Pala,9.3642,14.9046,89.10,37,71,5.48,TD,1616140123,,
252,Yambio,4.5721,28.3955,88.65,42,50,5.64,SS,1616140197,,
261,Douentza,14.9951,-2.9517,82.22,8,51,5.82,ML,1616140212,,
317,Port Pirie,-33.1833,138.0167,82.40,44,67,12.66,AU,1616140301,,
332,Broken Hill,-31.9500,141.4333,80.60,41,40,20.71,AU,1616140323,,


In [33]:
params= {"key": g_key,
        "radius": 5000,
         "types":"hotel",
        }

for index, row in clouds_df.iterrows():
    city=row['City']
    lat=row["Latitude"]
    lng=row['Longitude']
    
    params["location"]=f"{lat},{lng}"
    
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address= requests.get(base_url,params=params)
    name_address=name_address.json()
    
    print(json.dumps(name_address,indent=4, sort_keys=True))
    
    try:
        clouds_df.loc[index,"Hotel Name"]=name_address['results'][1]["name"]
#         clouds_df.loc[index,"Hotel Rating"]=name_address['results']["rating"]
    except(KeyError, IndexError):
        print("Missing field/results,,,skipping")
        time.sleep(1)
        

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKd5N3a_LVEA65HczxASQIMe0bgKWDcODK9QHCLnDoHXNdbKe7Fav0y8Sr-YMb4P7Tyby-FDwwM_Ks3ARFoTvXXCg3g3aFTcsS0gbonXw6pXEAiEGhPcRlXTAmqt4I-LCqXFUyE1NXAlbnT2bm0VQ6ePT4s3vAuduKrC2MaM3rBTTmJvlFL08k91uo7OXF0LlX2cmQZM3jv1VEJX6TylhWU8TtD5cMLPyYvFoOPlUdAGFW9LBXONpAjbMkAJPu8u87qfOHzoUrxGMJdavFJM5MBBFbTXpoHnuxPfvFInO0JiwCO8ZhYKWHCCvOZlNSHwGyvl5gR63VSp0xKjo33xx2Ayzy9OGpe8qKltBgoi8NYA48VVbUR7MXjF9zbAx8-AxGwPAwRKgWYn8eDb_vG1eRMFzDuYAlsh4IFRwCwuWK0R1bfAoVgBg",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -42.8826055,
                    "lng": 147.3257196
                },
                "viewport": {
                    "northeast": {
                        "lat": -42.65537552036029,
                        "lng": 147.6133832160388
                    },
                    "southwest": {
                        "lat": -43.01412256432244,
                        "lng": 147.1336604011

/Users/cooktherook94/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


{
    "html_attributions": [],
    "next_page_token": "ATtYBwJuxfD5Aa2WhDQ9p6i8ahJvY_NZuM2ppqn8zQk3Zxu1PDCQH3KbZ2Bm39Gj6gjkgvy_lTnYJEDwLeV14WLcxOGYA8p3QpvI-D0dsEBV3ak_G5pgo_bkQRBfjzAG0y9eFcf7N1V30DdP9BuJ5iGXovza5QF9L2RWrwZPLN5T36QHt4IRJy3NvVO8-0-UDb4Y5FWYka5p4aUCqkESIxoDTc5JTIana66Ub46SsA2PCAd3EcJwLHaxzHH0_NB2O6L1dJZUbAPjB_eDb7h5t5vDZHMBUWXyO_3Lnpz8wM3-B6Lc3AkG5TKW1B4YPj_piU-RUZr7EdxCCvhiJjwYTgPpHEmNjIiTur42AX6J7x4bDBnR1ANvLgQYzSBg6joqGcQEuVMtORvsqQkEPtCfZWeg-VR6qZCkSGoo4n7uxc99czUq1ATpa6ZVdA",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 12.6207821,
                    "lng": 80.1944915
                },
                "viewport": {
                    "northeast": {
                        "lat": 12.64652951397111,
                        "lng": 80.2059930328696
                    },
                    "southwest": {
                        "lat": 12.59812464084287,
                        "lng": 80.17629157331496
 

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLUxOtlEDDw2gqslJoorOO9C-b90HDXbWsRCCr-AyVvhEn_H61vyiBX10fjxyWQZGOUqviutrbzv22dwNBspe7avsiKiWISCVhKBcMOUEX2Q_6OlT2SInxQ2itKX3vrfII2_JuucI694w4bU8wt7GL2LudNQ3OM2w_JCcOHCSQ47oYnrzDp6auiFlYv16_jr2Rxi8JCUcQ3wI6A7OQXMtyMfzLsA_FesJWQiU3Ce3FFq0GTLb7bA1dOjfoI5FN8ErMqlFLUnlf-sz649xFoYY9SnWjmqugCyUK1PwsC4RCyHMfhacLH9CrqNatbe6OlSE5KtFdFJ2JJEer7qjbd9y3swgEkmux4Y63z81fQPSg_dJ2B85vgVo7x4GprzyUd6uHGmKHC_tb8ioSOMFFg3qLfb-5lsc571wAd7pXBQoNbRiyQWdIWPw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -33.9248685,
                    "lng": 18.4240553
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.47127004603382,
                        "lng": 19.00466996799566
                    },
                    "southwest": {
                        "lat": -34.35980610964359,
                        "lng": 18.307448775801

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIoqc_mG7c7dhP85ki3SdnSPqxCnZpfVYimGccBKrwzjsJ1szUZX-SDwAy2PBX-Ow3k4GXM3yjmmSfxASZBuzCQaiAwqq3hQW3TWZeUcZVnKX9TWxPd3l5G6ws4pgO0i9mOccIzmrU7TaViB6QXr6heTHmR-kW3dFv7WnPNgYH8Sm04pbK7sJjJR6f7erZ7umlC8vhG1fl6WJtaBLXBoTCjEuzOs4ZVpdbFuh9yDGeWSJuA5sbPTLOe2yaIeucbeu54LWTdimwzSwXw_bQyWGfCcLQVqr79ClcdEGVq1Ez0TnZrPf5yuJ8KxjEnI2v0NBSduQfAdD4WizhB3xvnli2gh2SXlPt6alyP6Fs8TA_kfUoP681zMdt3RaKDtqc40c95xjFMp7V3zHQGkaGeXKaPAhzGQG0Sj-18_uRi8PlzOBOs2FMgcw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 9.984756599999999,
                    "lng": 10.9510315
                },
                "viewport": {
                    "northeast": {
                        "lat": 9.990871192582267,
                        "lng": 10.95701682918237
                    },
                    "southwest": {
                        "lat": 9.980220367468581,
                        "lng": 10.94809063

...

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJxzCUT53gO28Mhe0kYzC2dwAupvhyPG_7ZuNPaPkleNiZIq1XzKAUpXJa8t9Ipm0sIwHGtxul6c1jJTc8A3S7088hQW0EKq9cf5gRAxPbMqpDwNTZWQKNKDsrQIm1QX0OmzuzntTLsH-EookwN4p5CyCRJLbKGpK-lpFvUOqxXUXghDDsLlBKq00nvwPTdYwMSk7Cd6WN1DTmC1IKW_-PUP42izg3tNmVDn4EvTbEnq7fzssNBnaAX_DfBFr0sSrLRXQc_BHgjtB9E8nViQAsw17NA89hoGCkfqKFfPl-qL512RlskP46b9OsTb8UMWdeRAPj6veuL3vqw6DdoDH1SepwkLTQwc6mg57RymCQ2SbyAT427jEGg29ybj58rRH7xnF1gw1BXvFq6tARi3MrB-omZvimj4SuDL4egkyopCvRbQWKBHw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 4.1228001,
                    "lng": 43.88880959999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 4.1276608293231,
                        "lng": 43.89638522473646
                    },
                    "southwest": {
                        "lat": 4.114067538215802,
                        "lng": 43.88175085018

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJdYZTnNeXoO-QtNQZUWRnyhLEUtb0HajGhd0Z-8MQ3L7VEBRDwvgNb91IyOX9cNzzwjVhoXyQs4JOZinHM0VeuZg_n8zjFMqnCaQX9J6EbypVcQ73-9BhhpxEbGT7QVcUMZcuN16d0aMhX4RfJAMZ8nfFBDUf5Wr1OBFA9wR21hRV_912DCGRwECrN5XQQKP1tqW3I7PcXSrTzrCkbPrQJ-98yU6aJAwe3XR-vbSPtXNLUQlmjXo3BKK3PLdwk2EK8tPkGerZom22re9Ioqw6ZFJXW_KVUNXwIJq_chmUT2ckUI9gnk9IXvxbWUbSW6vivqyMnDzAIJwVjsliT2Uxr_Qn59l3a9LrftAVYHBAI3qziQ4U2FGBwbv3nbIQyh0Q3jmG4eij7tzeUShWVk91KedF_aPkRgTChE4e4_frd1e0JZytZlQ",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -30.33240619999999,
                    "lng": 149.7812403
                },
                "viewport": {
                    "northeast": {
                        "lat": -30.02797928813836,
                        "lng": 150.0904286360803
                    },
                    "southwest": {
                        "lat": -30.42500631204713,
                        "lng": 149.462

{
    "html_attributions": [],
    "next_page_token": "ATtYBwI-wuH2fxGl_bD-x37vLBV1SBYIGJ8_0uaPazT8jMmMKRfilx7Y4AeyHnF5D1_PA3rWnVj5Ya6KV8Ow9e2xsQ_LVal4pKoyJ-6M2Bdnz1FI8eLl0onlAJML1WRIVDomuuvm4FsdSix_AcPEbT5pZX6xM0oJHnYvywvaVnflb0RmMi3fc2UwAYKnJs-2dBJdV0aPYDKw7AMKY9TqFNq8p4JN6dS8Z8qI1z9wCL_QnMNenYm9wiw5OduReu1e4h_3xRYbt9d71QgOQqMVqBG3OYabkAcpv9sYLrVF4Lbe8D4LCjEfhO4h1hWjyA_vy9xPN1Hp12OIrTCGFjrGKirxDQDJipToDIhLh9YOQh-_QcE07ycbeX1LXCuD9lEs3H2AqRGQYTQxpp5M9_XVZwHzBWe0xYDmAjzQn1OdFaG2rBKPc7ZGT55rHQ",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 32.5326308,
                    "lng": 20.5870955
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.54659602276338,
                        "lng": 20.60159677693943
                    },
                    "southwest": {
                        "lat": 32.51851948691252,
                        "lng": 20.55301673627194


{
    "html_attributions": [],
    "next_page_token": "ATtYBwJ7Rl_Jp3TmdX-nTYCYEp8Xwk0-ML5rhehxtroImqRAmErjxm6qqLsH5cKEYLrj5MRK_iDgdS8BNJmG7-AkjGZaZD6FvxLobLhAR82BbVCBOwv2yN5KPQylW44xVVDRyk6fAERA70fJnctsbzUBtv8RxO6S1HtZV2II6kqSZ9w8qlX81fiXTMrZ-b54ub7PIxFRvyyCi9BFd0Cag-oE7phcZs7HJdeY47fyRuthmIp-azNp5ZBdHvZQ1L7qNV8gN-IegYi1YcA17HfdNwucHeIltyYpXaFhi5fqEhGB5fK3HKkKZEnd0qisCByT2mB_oUgVkZujCYEwQT8c562BHu1413jYbchWsCxmyDuy74FgIyg65YWp4FgnwYnBzaenpYJ46ERLA0HuAiCqCqjONCSafj-qYTqFFKt-idMdrPUM6S2Lcdic-A",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 34.16818140000001,
                    "lng": 71.7503855
                },
                "viewport": {
                    "northeast": {
                        "lat": 34.20659728846233,
                        "lng": 71.78106371879967
                    },
                    "southwest": {
                        "lat": 34.12992703413464,
                        "lng": 71.71458465

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJZneYJv_PeWPjOlRklvVc4jT8y5h-d2JxYaYfgV5_BBKysclB4R5qb2ny4Aw0TOMBws_1kyfv7ujIiEOOm5hESEH9dFM6iq-65VwhAge-wYx037by2qbjK0oEYNsVZgO4yEfo4_5jP_6xJgj7jBiT9MwTOEQqGM2cYWz_aZsmlAbP96zC8QBEz2ZjhHKXI0E39KekoanN_z_vI3ICTsdYoKN-Wp_TcFV4az5IUx1fNZdZyq00IJQ6Eqte8XWPOW03iDEkLLTnBBotMZKAYhBdIZx_JezGzK6SahWvQ59EUUOIRJ1rE71bReLb2TU8VsM3nZcUv8RGHU0UPMOBhyTGkNY4dWXjXOS9z-qychCPOBMQCfXKxgJuKcOXSo2U80WfJ_PYDqQftJk2Y3YZ_KrKo00wuhsvD_LD8jbEJefzHei23nD8KRg",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 9.3802886,
                    "lng": 80.3769999
                },
                "viewport": {
                    "northeast": {
                        "lat": 9.417352926767059,
                        "lng": 80.43511396355237
                    },
                    "southwest": {
                        "lat": 9.3365196607499,
                        "lng": 80.31809482679745
   

In [34]:
hotel_df

,City,Latitude,Longitude,Max_temp,Humidity,Cloudiness,Wind_speed,Country,Date,Hotel Name,Hotel Rating
10,Hobart,-42.8794,147.3294,69.80,49,20,11.50,AU,1616139813,Mantra on Collins Hobart,
47,Mahabalipuram,12.6264,80.1722,87.80,55,40,8.05,IN,1616139884,Radisson Blu Resort Temple Bay Mamallapuram,
73,Cape Town,-33.9258,18.4232,66.20,59,20,20.71,ZA,1616139684,Southern Sun Waterfront Cape Town,
177,Pindiga,9.9833,10.9333,84.36,52,75,12.30,NG,1616140079,MTN - Bukuru Connect Point,
186,Hong Kong,22.2855,114.1577,82.40,54,40,6.91,HK,1616140037,Mini Hotel Central,
205,Pala,9.3642,14.9046,89.10,37,71,5.48,TD,1616140123,Pala Urbain Centre de Santé,
252,Yambio,4.5721,28.3955,88.65,42,50,5.64,SS,1616140197,Western Equatoria State,
261,Douentza,14.9951,-2.9517,82.22,8,51,5.82,ML,1616140212,Marché de Douentza,
317,Port Pirie,-33.1833,138.0167,82.40,44,67,12.66,AU,1616140301,"PORT PIRIE, ED",
332,Broken Hill,-31.9500,141.4333,80.60,41,40,20.71,AU,1616140323,Royal Exchange Hotel,


In [70]:
# query_url = f'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel%20{city}&inputtype=textquery&&key={gkey}'

In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "'Hotel Name'"

In [ ]:
# Add marker layer ontop of heat map


# Display figure
